# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2000 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2000 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.


In [98]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

IN_FILE  = "strawberry-prices.tsv"       # This is provided
OUT_FILE = "strawberry-backtest.tsv"     # This is what I create

df = pd.read_csv(IN_FILE, sep="\t")

df["month"] = pd.to_datetime(df["month"]).dt.to_period("M")

train = df[df["month"] <= pd.Period("2023-12", freq="M")].copy()
bt_months = pd.period_range("2024-01", "2024-12", freq="M")

def make_design(frame, base_start, center_val):

    t = ((frame["month"].dt.year - base_start.year) * 12 +
         (frame["month"].dt.month - base_start.month)).astype(int).to_numpy() 

    if center_val is None:
        tc = t - t.mean()
    else:
        tc = t - center_val

    tc= t - center_val

    t2 = tc**2
    t3 = tc**3

    m = frame["month"].dt.month
    md = pd.get_dummies(m)                                  
    md = md.reindex(columns=range(1, 13), fill_value=0)      
    names = {i: ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"][i-1] for i in range(1,13)}
    md = md.rename(columns=names)
    md = md.drop(columns=["Dec"], errors="ignore")         
    M = md.to_numpy(dtype=float)

    const = np.ones((len(frame), 1))
    X = np.hstack([const, tc[:, None], t2[:, None], M])
    y = frame["price"].to_numpy(dtype=float) if "price" in frame.columns else None
    return X, y

base = train["month"].min()    
t_train = ((train["month"].dt.year - base.year) * 12 +
           (train["month"].dt.month - base.month)).astype(int).to_numpy()
center = t_train.mean()

X_tr, y_tr = make_design(train, base, center) 
beta, *_ = np.linalg.lstsq(X_tr, y_tr, rcond=None)

bt_frame = pd.DataFrame({"month": bt_months})
X_bt, _ = make_design(bt_frame, base, center)
pred_price = X_bt @ beta

out = pd.DataFrame({
    "month": bt_months.to_timestamp(),  # Timestamp at month start
    "price": pred_price
})

out["month"] = out["month"].dt.strftime("%Y-%m-%d")

out.to_csv(OUT_FILE, sep="\t", index=False)

print(f" {OUT_FILE} with shape {out.shape}")
display(out.head(3))
display(out.tail(3))

chk = pd.read_csv(OUT_FILE, sep="\t")
assert list(chk.columns) == ["month", "price"]
assert len(chk) == 12



 strawberry-backtest.tsv with shape (12, 2)


,month,price
0,2024-01-01,4.575410
1,2024-02-01,4.271944
2,2024-03-01,3.837477


,month,price
9,2024-10-01,3.850960
10,2024-11-01,4.290994
11,2024-12-01,4.649602


Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth? (If your mean is not close to zero, then you may be missing a long term trend.)

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [99]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

ACT_FILE = "strawberry-prices.tsv"      
BT_FILE  = "strawberry-backtest.tsv"   
OUT_FILE = "backtest-accuracy.tsv"

truth = pd.read_csv(ACT_FILE, sep="\t")
truth["month"] = pd.to_datetime(truth["month"])
truth_2024 = truth[truth["month"].dt.year == 2024][["month", "price"]].rename(columns={"price":"actual"})

bt = pd.read_csv(BT_FILE, sep="\t")
bt["month"] = pd.to_datetime(bt["month"])
bt = bt.rename(columns={"price":"pred"})

df = truth_2024.merge(bt, on="month", how="inner")
assert len(df) == 12, f"Expected 12 months, got {len(df)}"

df["resid"] = df["actual"] - df["pred"]
mean_resid = float(df["resid"].mean())
std_resid  = float(df["resid"].std(ddof=1))  

#to save the results 
pd.DataFrame([{"mean": mean_resid, "std": std_resid}]).to_csv(OUT_FILE, sep="\t", index=False)
print(f" {OUT_FILE} with mean={mean_resid:.6f}, std={std_resid:.6f}")
df.head()



 backtest-accuracy.tsv with mean=-0.041700, std=0.258429


,month,actual,pred,resid
0,2024-01-01,5.055,4.575410,0.479590
1,2024-02-01,4.264,4.271944,-0.007944
2,2024-03-01,3.742,3.837477,-0.095477
3,2024-04-01,3.576,3.903960,-0.327960
4,2024-05-01,3.237,3.611044,-0.374044


Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2000 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.

In [100]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

#this is the same one as part 1 
IN_FILE  = "strawberry-prices.tsv"
OUT_FILE = "strawberry-forecast.tsv"

df = pd.read_csv(IN_FILE, sep="\t", parse_dates=["month"]).sort_values("month").reset_index(drop=True)

train_all = df[df["month"].dt.year <= 2024].copy()

def make_design(frame, base_start):
    m = frame["month"]

    # Jan is the baseline
    d = pd.get_dummies(m.dt.month, drop_first=True)
    d = d.reindex(columns=range(2, 13), fill_value=0)  

    t = ((m.dt.year - base_start.year) * 12 + (m.dt.month - base_start.month)).astype(float)

    X = np.hstack([
        np.ones((len(frame), 1)),        
        d.to_numpy(),                    
        t.to_numpy().reshape(-1, 1)     
    ])
    y = frame["price"].to_numpy(float)
    return X, y

base = train_all["month"].min()
X_all, y_all = make_design(train_all, base)
beta, *_ = np.linalg.lstsq(X_all, y_all, rcond=None)

fcst_months = pd.date_range("2025-01-01", "2025-12-01", freq="MS")
d25 = pd.get_dummies(fcst_months.month, drop_first=True).reindex(columns=range(2, 13), fill_value=0)
t25 = ((fcst_months.year - base.year) * 12 + (fcst_months.month - base.month)).astype(float)

X25 = np.hstack([
    np.ones((len(fcst_months), 1)),
    d25.to_numpy(),
    t25.to_numpy().reshape(-1, 1)
])

y25 = X25 @ beta
out = pd.DataFrame({"month": fcst_months, "price": y25})
out.to_csv(OUT_FILE, sep="\t", index=False)

chk = pd.read_csv(OUT_FILE, sep="\t")
assert list(chk.columns) == ["month", "price"]
assert len(chk) == 12
print(f" {OUT_FILE} with {len(out)} rows.")


 strawberry-forecast.tsv with 12 rows.


Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [101]:
# YOUR CHANGES HERE
import pandas as pd
import numpy as np

FORECAST_FILE = "strawberry-forecast.tsv"
OUT_FILE      = "timings.tsv"

BUDGET = 1_000_000            
FREEZE_COST = 0.20            
STORAGE_COST_PER_MONTH = 0.10 
SELL_DISCOUNT = 0.10          # 10% discount 

# Load 2025 forecast
fc = pd.read_csv(FORECAST_FILE, sep="\t", parse_dates=["month"]).sort_values("month").reset_index(drop=True)

rows = []
n = len(fc)

for i in range(n):
    for j in range(i + 1, n): 
        buy_month  = fc.loc[i, "month"]
        sell_month = fc.loc[j, "month"]
        buy_price  = float(fc.loc[i, "price"])
        sell_price = float(fc.loc[j, "price"])

        months_gap = (sell_month.year - buy_month.year) * 12 + (sell_month.month - buy_month.month)

        per_pint_cost = buy_price + FREEZE_COST + STORAGE_COST_PER_MONTH * months_gap

        pints = int(np.floor(BUDGET / per_pint_cost)) if per_pint_cost > 0 else 0

        per_pint_profit = sell_price * (1 - SELL_DISCOUNT) - per_pint_cost

        expected_profit = pints * per_pint_profit

        rows.append({
            "buy_month":  buy_month.strftime("%Y-%m-%d"),
            "sell_month": sell_month.strftime("%Y-%m-%d"),
            "pints_purchased": pints,
            "expected_profit": expected_profit
        })

timings = pd.DataFrame(rows, columns=["buy_month", "sell_month", "pints_purchased", "expected_profit"])
timings.to_csv(OUT_FILE, sep="\t", index=False)

chk = pd.read_csv(OUT_FILE, sep="\t")
assert list(chk.columns) == ["buy_month", "sell_month", "pints_purchased", "expected_profit"]
print(f" {OUT_FILE} with {len(timings)} rows (all buy→sell pairs for 2025).")


 timings.tsv with 66 rows (all buy→sell pairs for 2025).


Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns best_profit and one_std_profit.

In [102]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

TIMINGS_FILE   = "timings.tsv"
FORECAST_FILE  = "strawberry-forecast.tsv"
ACCURACY_FILE  = "backtest-accuracy.tsv"
OUT_FILE       = "check.tsv"

# This is the same as Part 4
BUDGET = 1_000_000
FREEZE_COST = 0.20
STORAGE_COST_PER_MONTH = 0.10
SELL_DISCOUNT = 0.10  # 10%

# inputs
timings = pd.read_csv(TIMINGS_FILE, sep="\t", parse_dates=["buy_month","sell_month"])
fc      = pd.read_csv(FORECAST_FILE, sep="\t", parse_dates=["month"])
acc     = pd.read_csv(ACCURACY_FILE, sep="\t")
std_resid = float(acc.loc[0, "std"])

best = timings.loc[timings["expected_profit"].idxmax()].copy()
buy_month  = pd.to_datetime(best["buy_month"])
sell_month = pd.to_datetime(best["sell_month"])
pints_best = int(best["pints_purchased"])

prices = fc.set_index("month")["price"]
buy_price  = float(prices.loc[buy_month])
sell_price = float(prices.loc[sell_month])

months_gap = (sell_month.year - buy_month.year) * 12 + (sell_month.month - buy_month.month)
per_pint_cost = buy_price + FREEZE_COST + STORAGE_COST_PER_MONTH * months_gap

per_pint_profit_best = sell_price * (1 - SELL_DISCOUNT) - per_pint_cost
best_profit = pints_best * per_pint_profit_best

sell_price_down = sell_price - std_resid
per_pint_profit_down = sell_price_down * (1 - SELL_DISCOUNT) - per_pint_cost
one_std_profit = pints_best * per_pint_profit_down

out = pd.DataFrame(
    [{"best_profit": best_profit, "one_std_profit": one_std_profit}],
    columns=["best_profit", "one_std_profit"]
)
out.to_csv(OUT_FILE, sep="\t", index=False)

chk = pd.read_csv(OUT_FILE, sep="\t")
assert list(chk.columns) == ["best_profit", "one_std_profit"]
print(f" {OUT_FILE}. Best profit ≈ {best_profit:,.2f}, one-std-down profit ≈ {one_std_profit:,.2f}.")



 check.tsv. Best profit ≈ 101,434.80, one-std-down profit ≈ 45,087.93.


Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


In [103]:
from textwrap import dedent

ack = dedent("""\
Acknowledgments


The Libraries/Tools used 
- pandas for data loading and manipulation
- numpy for numeric calculations
- matplotlib plots for sanity checks

Data
- I used the course provided file strawberry-prices.tsv

People/Collaboration
- I discussed this assignment with no one and did not use any additional sources beyond the course materials,
other than occasional Google searches to help understand concepts covered in class. 

""")

with open("acknowledgments.txt", "w", encoding="utf-8") as f:
    f.write(ack)

print(" acknowledgments.txt")

 acknowledgments.txt


Submit "acknowledgements.txt" in Gradescope.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.